In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

from helper import readFile
from helper import oneHotEncoder
from helper import showConfusionMarix
from helper import getBatches
from NeuralNetwork import NeuralNetwork

%matplotlib inline

In [2]:
# read the data
trainFeature, trainLabel = readFile('./data/train/lab/hw2train_labels.txt', './data/')
devFeature, devLabel = readFile('./data/dev/lab/hw2dev_labels.txt', './data/')
evalFeature, evalLabel = readFile('./data/eval/lab/hw2eval_labels.txt', './data/')

# One-Hot-Encode for labels
trainLabel = oneHotEncoder(trainLabel, 9)
devLabel = oneHotEncoder(devLabel, 9)
evalLabel = oneHotEncoder(evalLabel, 9)

## TensorFlow

In [3]:
#########################################################
# Part 2: TensorFlow Analysis
print('*' * 60)
print('Part 2: TensorFlow Analysis')

************************************************************
Part 2: TensorFlow Analysis


In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [5]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], 
                          strides=[1, 2, 2, 1], padding='SAME')

## TDNN model

In [16]:
batchSize = 50
subIterate = 5

trainAcc = []
devAcc = []

In [15]:
# begin the session
sess = tf.InteractiveSession()

# Create the model
x = tf.placeholder(tf.float32, shape=[None, 1120])
y_ = tf.placeholder(tf.float32, shape=[None, 9])

# first convolutional layer
W_conv1 = weight_variable([5, 5, 1, 8])
b_conv1 = bias_variable([8])

# reshape the input image
x_image = tf.reshape(x, [-1, 70, 16, 1])

# first processing
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

# second convolution layer
W_conv2 = weight_variable([5, 5, 8, 3])
b_conv2 = bias_variable([3])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)

# Densely Connected Layer
W_fc1 = weight_variable([70 * 16 * 3, 100])
b_fc1 = bias_variable([100])

h_conv2_flat = tf.reshape(h_conv2, [-1, 70 * 16 * 3])
h_fc1 = tf.nn.relu(tf.matmul(h_conv2_flat, W_fc1) + b_fc1)

# readout layer
W_fc2 = weight_variable([100, 9])
b_fc2 = bias_variable([9])

y_conv=tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)

# train and test the model
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())

for i in range(100):
    X, y = shuffle(trainFeature, trainLabel)
    batchesX, batchesY = getBatches(X, y, batchSize)
    
    if i%10 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:trainFeature, y_: trainLabel})
        dev_accuracy = accuracy.eval(feed_dict={x:devFeature, y_: devLabel})
        eval_accuracy = accuracy.eval(feed_dict={x:evalFeature, y_: evalLabel})
        print("Step %d, train, dev, eval accuracy \t%g, %g, %g,"%(i, np.round(train_accuracy, 5), 
                                                                  np.round(dev_accuracy, 5), 
                                                                  np.round(eval_accuracy, 5)))
    # training process
    j = 1
    for batchFeature, batchLabel in zip(batchesX, batchesY):
        if j == subIterate:
            break
        train_step.run(feed_dict={x: batchFeature, y_: batchLabel})
        j += 1
        
print("test accuracy %g"%accuracy.eval(feed_dict={x: evalFeature, y_: evalLabel}))

sess.close()

Step 0, train, dev, eval accuracy 	0.10198, 0.10397, 0.10341,


KeyboardInterrupt: 

In [10]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))